# Fetch Text from PDF

In [ ]:
%pip install pdfplumber pytesseract pdf2image

In [ ]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        
        if text.strip():
            return text.strip()
    except Exception as e:
        print(f"Error reading PDF file {pdf_path}: {e}")

    print("Falling back to OCR for image-based PDFs.")
    try:
        images =    convert_from_path(pdf_path)
        for image in images:
            text += pytesseract.image_to_string(image) + "\n"
    except Exception as e:
        print(f"Error during OCR processing of {pdf_path}: {e}")
    
    return text.strip()

In [ ]:
pdf_path = "Resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)

print("\nExtracted Text from PDF:")
print(resume_text if resume_text else "No text could be extracted from the PDF.")  

# Google GenAI API Key

In [ ]:
%pip install google-generativeai python-dotenv

In [ ]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
print("GOOGLE_API_KEY:", os.getenv("GOOGLE_API_KEY"))
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash")

In [ ]:
response = model.generate_content("What is the capital of France?")

In [ ]:
print(response)

In [ ]:
print(response.text)

# Resume Analysis

In [ ]:
def analyze_resume(resume_text, job_description = None):
    if not resume_text:
        return {"error": "No resume text provided."}
    
    model = genai.GenerativeModel("gemini-2.5-flash")

    basic_prompt = f"""
    You are an expert resume analyzer. Given the following resume text and job description, provide a detailed analysis of how well the resume matches the job requirements.

    Resume:
    {resume_text}
    """

    if job_description:
        basic_prompt += f"""
        Additionally, consider the following job description:

        Job Description:
        {job_description}

        Highlight the key skills, experiences, and qualifications that make the candidate a good fit for the job.
        """
    response = model.generate_content(basic_prompt)

    analysis = response.text.strip()
    return analysis

In [ ]:
print(analyze_resume(resume_text))